# Graded Assignment on Monitoring, Scaling and Automation

#### Develop a system that automatically manages the lifecycle of a web application hosted on  EC2 instances, monitors its health, and reacts to changes in traffic by scaling resources.  Furthermore, administrators receive notifications regarding the infrastructure's health and scaling events

#### Task 1

##### 1. Web Application Deployment: 
 - Use `boto3` to: 
 
 - Create an S3 bucket to store your web application's static files. 


In [ ]:
# Function to create an S3 bucket
def create_bucket(bucket_name):
    s3 = boto3.client('s3')
    try:
        res = s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={
                'LocationConstraint': 'ap-northeast-2'
            }
        )
        print("Bucket created:", res)
        return res
    except Exception as e:
        print("Error creating bucket:", e)

# Function to upload files to S3 bucket
def uploadfolders(bucket_name, folder_path):
    s3 = boto3.client('s3')
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            s3.upload_file(os.path.join(root, file), bucket_name, file)
            print("Uploaded:", file)

# Main process function to create bucket and upload files
def process():
    bucket_name = "monitorbeluga"
    folder_path = "C:/Users/eagleye/Desktop/FOCUS/Samplefiles"
    create_bucket(bucket_name)
    uploadfolders(bucket_name, folder_path)

#### Task 1.2

 - Launch an EC2 instance and configure it as a web server (e.g., Apache, Nginx).  - Deploy the web application onto the EC2 instance. 

In [ ]:
def create_ec2_instance():
    ec2 = boto3.resource('ec2')
    instance = ec2.create_instances(
        ImageId='ami-062cf18d655c0b1e8',  # Use a valid AMI ID
        MinCount=1,
        MaxCount=1,
        InstanceType='t2.micro',
        KeyName='Gani_reborne',  # Make sure you have this key pair created
        TagSpecifications=[
            {
                'ResourceType': 'instance',
                'Tags': [
                    {
                        'Key': 'Name',
                        'Value': 'EagleEye'
                    }
                ]
            }
        ],
        UserData='''#!/bin/bash
                    sudo apt update -y
                    sudo apt install nginx -y
                    service nginx start
                    mkdir -p /var/www/uploads
                    crontab -l | { cat; echo "* * * * * aws s3 sync /var/www/uploads s3://monitorbeluga/uploads"; } | crontab -
                    aws s3 cp s3://monitorbeluga /var/www/html --recursive
'''
    )
    instance[0].wait_until_running()
    instance[0].reload()
    print(instance)
    security_groups = instance[0].security_groups
    print("Security Groups:", security_groups)
    print("Security Groups:", [sg['GroupName'] for sg in security_groups])
    print("EC2 Instance created:", instance[0].id)
    return instance[0]

#### Task2

##### 2. Load Balancing with ELB: 

 - Deploy an Application Load Balancer (ALB) using `boto3`. 

 - Register the EC2 instance(s) with the ALB. 

In [ ]:
def create_target_group(vpc_id):
    elbv2 = boto3.client('elbv2')
    response = elbv2.create_target_group(
        Name='EagleEyeTG',
        Protocol='HTTP',
        Port=80,
        VpcId=vpc_id,
        HealthCheckProtocol='HTTP',
        HealthCheckPort='80',
        HealthCheckEnabled=True,
        HealthCheckPath='/',
        HealthCheckIntervalSeconds=30,
        HealthCheckTimeoutSeconds=5,
        HealthyThresholdCount=5,
        UnhealthyThresholdCount=2,
        TargetType='instance',
    )
    target_group_arn = response['TargetGroups'][0]['TargetGroupArn']
    print("Target Group created:", target_group_arn)
    return target_group_arn

# Function to create a load balancer and attach it to the target group
def attach_load_balancer(target_group_arn):
    ec2 = boto3.resource('ec2', region_name='ap-northeast-2')
    client = boto3.client('ec2', region_name='ap-northeast-2')

    # Fetch the default VPC
    default_vpc = list(ec2.vpcs.filter(Filters=[{'Name': 'isDefault', 'Values': ['true']}]))[0]

    # Fetch all subnets in the specified availability zones
    specified_subnets = client.describe_subnets(
        Filters=[
            {'Name': 'availability-zone', 'Values': ['ap-northeast-2a', 'ap-northeast-2b']}
        ]
    )['Subnets']

    if len(specified_subnets) < 2:
        raise ValueError("At least two subnets are required in the specified availability zones.")

    # Fetch subnets in the default VPC
    default_vpc_subnets = list(default_vpc.subnets.all())

    if not default_vpc_subnets:
        raise ValueError("No subnets found in the default VPC.")

    # Combine the subnets (one from default VPC and two specified subnets)
    combined_subnets = [default_vpc_subnets[0].id] + [subnet['SubnetId'] for subnet in specified_subnets[:2]]

    elbv2 = boto3.client('elbv2', region_name='ap-northeast-2')
    response = elbv2.create_load_balancer(
        Name='EagleEyeELB',
        Scheme='internet-facing',
        Subnets=combined_subnets,  
        Tags=[
            {
                'Key': 'Name',
                'Value': 'EagleEye',
            },
        ]
    )
    load_balancer_arn = response['LoadBalancers'][0]['LoadBalancerArn']
    listener = elbv2.create_listener(
        LoadBalancerArn=load_balancer_arn,
        Protocol='HTTP',
        Port=80,
        DefaultActions=[
            {
                'Type': 'forward',
                'TargetGroupArn': target_group_arn,
            }
        ]
    )
    print("Load Balancer created and listener attached:", load_balancer_arn)
    return load_balancer_arn

# Function to register targets to the target group
def register_targets(target_group_arn, instance_id):
    elbv2 = boto3.client('elbv2')
    response = elbv2.register_targets(
        TargetGroupArn=target_group_arn,
        Targets=[
            {
                'Id': instance_id,
                'Port': 80
            },
        ]
    )
    print("Instance registered to Target Group:", response)
    return response


#### Task 3
##### 3. Auto Scaling Group (ASG) Configuration: 

 - Using `boto3`, create an ASG with the deployed EC2 instance as a template. 

 - Configure scaling policies to scale in/out based on metrics like CPU utilization or network traffic.


In [ ]:
def create_launch_configuration():
    asg = boto3.client('autoscaling')
    launch_config_name = 'EagleEyeLaunchConfig'
    try:
        response = asg.create_launch_configuration(
            LaunchConfigurationName=launch_config_name,
            ImageId='ami-062cf18d655c0b1e8',
            InstanceType='t2.micro',
            KeyName='Gani_reborne',
            SecurityGroups=['default']
        )
        print("Launch Configuration created:", response)
        return response
    except asg.exceptions.AlreadyExistsFault:
        print(f"Launch Configuration {launch_config_name} already exists.")
        return None

# Function to create an Auto Scaling Group (ASG)
def create_autoscaling_group():
    asg = boto3.client('autoscaling')
    create_launch_configuration()
    try:
        response = asg.create_auto_scaling_group(
            AutoScalingGroupName='EagleEyeASG',
            LaunchConfigurationName='EagleEyeLaunchConfig',
            MinSize=1,
            MaxSize=3,
            DesiredCapacity=1,
            DefaultCooldown=300,
            AvailabilityZones=[
                'ap-northeast-2a',
                'ap-northeast-2b',
            ],
            Tags=[
                {
                    'Key': 'Name',
                    'Value': 'EagleEye',
                    'PropagateAtLaunch': True
                },
            ]
        )
        print("Auto Scaling Group created:", response)

        # Set up notification configuration
        sns_topic_arn = create_sns_topic('EagleEyeScalingEvents')
        asg.put_notification_configuration(
            AutoScalingGroupName='EagleEyeASG',
            TopicARN=sns_topic_arn,
            NotificationTypes=[
                'autoscaling:EC2_INSTANCE_LAUNCH',
                'autoscaling:EC2_INSTANCE_TERMINATE'
            ]
        )
        print("Notification configuration added.")
        return response
    except Exception as e:
        print("Error creating Auto Scaling Group:", e)


#### Task 4

##### 4. SNS Notifications: 

 - Set up different SNS topics for different alerts (e.g., health issues, scaling events, high traffic). 

 - Integrate SNS with Lambda so that administrators receive SMS or email notifications. 

In [ ]:
def create_sns_topic(name):
    sns = boto3.client('sns')
    response = sns.create_topic(Name=name)
    print("SNS Topic created:", response['TopicArn'])
    return response['TopicArn']

# Function to create a Lambda function
def create_lambda_function():
    lambda_client = boto3.client('lambda')
    try:
        response = lambda_client.create_function(
            FunctionName='EagleEyeNotificationHandler',
            Runtime='python3.8',
            Role='arn:aws:iam::123456789012:role/YourLambdaExecutionRole',  # Provide the appropriate Lambda execution role ARN
            Handler='lambda_function.lambda_handler',
            Code={
                'ZipFile': open('lambda_function.zip', 'rb').read(),  # Make sure you have your Lambda function code packaged
            },
            Description='Lambda function to handle SNS notifications',
            Timeout=15,
            MemorySize=128,
            Publish=True,
        )
        print("Lambda Function created:", response['FunctionArn'])
        return response['FunctionArn']
    except Exception as e:
        print("Error creating Lambda function:", e)

# Function to subscribe Lambda to SNS topic
def subscribe_lambda_to_sns(topic_arn, lambda_arn):
    sns = boto3.client('sns')
    response = sns.subscribe(
        TopicArn=topic_arn,
        Protocol='lambda',
        Endpoint=lambda_arn
    )
    print("Lambda subscribed to SNS Topic:", response)
    return response

# Function to create a Lambda function for moving files from EC2 to S3
def create_file_mover_lambda_function():
    lambda_client = boto3.client('lambda')
    try:
        response = lambda_client.create_function(
            FunctionName='EagleEyeFileMover',
            Runtime='python3.8',
            Role='arn:aws:iam::975050024946:user/saiganesh7989@gmail.com',  # Provide the appropriate Lambda execution role ARN
            Handler='file_mover.lambda_handler',
            Code={
                'ZipFile': open('file_mover.zip', 'rb').read(),  # Make sure you have your Lambda function code packaged
            },
            Description='Lambda function to move files from EC2 to S3',
            Timeout=60,
            MemorySize=128,
            Publish=True,
        )
        print("Lambda Function created:", response['FunctionArn'])
        return response['FunctionArn']
    except Exception as e:
        print("Error creating Lambda function:", e)

# Create the Lambda function code for handling notifications
lambda_notification_code = '''
import json
import boto3

def lambda_handler(event, context):
    sns_message = event['Records'][0]['Sns']['Message']
    print("Received SNS message:", sns_message)
    
    # Initialize the SNS client
    sns_client = boto3.client('sns')
    
    # Define the recipient's phone number
    recipient_phone_number = '9182345999'  # Replace with actual phone number
    
    # Logic to handle different types of notifications
    if 'health issue' in sns_message:
        # Handle health issue notification
        sns_client.publish(
            PhoneNumber=recipient_phone_number,
            Message=f"Health Issue Alert: {sns_message}"
        )
    elif 'scaling event' in sns_message:
        # Handle scaling event notification
        sns_client.publish(
            PhoneNumber=recipient_phone_number,
            Message=f"Scaling Event Alert: {sns_message}"
        )
    elif 'high traffic' in sns_message:
        # Handle high traffic notification
        sns_client.publish(
            PhoneNumber=recipient_phone_number,
            Message=f"High Traffic Alert: {sns_message}"
        )
    else:
        # Handle general notifications
        sns_client.publish(
            PhoneNumber=recipient_phone_number,
            Message=f"General Alert: {sns_message}"
        )
'''

# Write the Lambda function code to a file
with open('lambda_function.py', 'w') as f:
    f.write(lambda_notification_code)

# Zip the Lambda function code
with zipfile.ZipFile('lambda_function.zip', 'w') as z:
    z.write('lambda_function.py')

# Clean up the temporary files
os.remove('lambda_function.py')

# Create the Lambda function code for moving files from EC2 to S3
lambda_file_mover_code = '''
import os
import boto3

s3 = boto3.client('s3')

def lambda_handler(event, context):
    bucket_name = 'monitorbeluga'
    source_directory = '/var/www/uploads'
    for filename in os.listdir(source_directory):
        local_path = os.path.join(source_directory, filename)
        if os.path.isfile(local_path):
            s3.upload_file(local_path, bucket_name, f'uploads/{filename}')
            os.remove(local_path)
    print("Files moved to S3")
'''

# Write the Lambda function code to a file
with open('file_mover.py', 'w') as f:
    f.write(lambda_file_mover_code)

# Zip the Lambda function code
with zipfile.ZipFile('file_mover.zip', 'w') as z:
    z.write('file_mover.py')

# Clean up the temporary files
os.remove('file_mover.py')


#### Task 5

##### 5. Infrastructure Automation: 

 - Create a single script using `boto3` that: 

 - Deploys the entire infrastructure. 

 - Updates any component as required. 

 - Tears down everything when the application is no longer needed. 